In [ ]:
import numpy as np
import time
import pandas as pd
import openpyxl
import os
from qiskit import QuantumCircuit, transpile

from Noise import QiskitNoiseModelDepol,QiskitNoiseModelBitflip, QiskitNoiseModelAmplitudeDamp
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator
from qiskit.utils.parallel import parallel_map
from qiskit_aer.noise import NoiseModel

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeMarrakesh

#
# Helper functions
#
def groverSuboptimalIterations(n, k):
    return int(np.round(0.58278 * np.sqrt(n / k)))

# n is the size of the database, m is the number of marked items
def grover_iterations(m, n):
    #return int(np.round(np.pi *(np.sqrt(n)/4)))

    num =int(np.round(np.pi / (4 * np.arcsin(np.sqrt(m / n))) - 1 / 2))
    if num<1:
        return 1
    else:
        return num

# selects n unique random numbers from in the range [0 - upper_bound)
def select_random_elements(upper_bound, n):
    elements = list(range(upper_bound))
    selected = []
    for i in range(n):
        index = np.random.randint(len(elements))
        selected.append(elements[index])
        del elements[index]
    return selected

# Checks if the integer representetion of binary_string exists in number_list
def binary_string_in_list(number_list, binary_string):
    num = int(binary_string, 2)
    return num in number_list

#
# Grover implementation
#

def phase_oracle(n, indicies_to_mark, name='Oracle'):
    qc = QuantumCircuit(n, name=name)
    oracle_matrix = np.identity(2**n)
    for index_to_mark in indicies_to_mark:
        oracle_matrix[index_to_mark, index_to_mark] = -1
    qc.unitary(Operator(oracle_matrix), range(n))
    return qc


# Dynamic programming optimization of diffuser function
# dict to save dicts that have been computed 
diffuser_dict = {}

def diffuser(n):
    if n not in diffuser_dict:
        qc = QuantumCircuit(n, name='Diffuser')
        qc.h(range(n))
        qc.append(phase_oracle(n, [0]), range(n))
        qc.h(range(n))
        diffuser_dict.update({n: qc})
    return diffuser_dict.get(n)


def Grover(n, marked, r):
    # Reuse the same oracle instance rather than creating new ones
    qc_phase = phase_oracle(n, marked)
    qc_diffuser = diffuser(n)
    
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    for _ in range(r):
        qc.append(qc_phase, range(n)) 
        qc.append(qc_diffuser, range(n))
    qc.measure(range(n), range(n))
    return qc

#
# Functions to run experiments
#
def simulator_experiment_noise(n, noise_level):
    start = time.time()

    nums = range(1, int(np.floor((2**n) / 2)) + 1)

    # Fewer processes for larger circuits to reduce memory contention
    num_processes = max(1, 4 - (n - 3))  # Scale down as n increases
    results = parallel_map(experiment_parallel_noise, nums, 
                         task_args=([n, noise_level]), 
                         num_processes=num_processes)

    end = time.time()
    saving_data(n, noise_level, results)
    print(f"Time taken: {end - start:.2f} seconds")

def saving_data(n, noise_level, results):
    # Extract actual probabilities and oracle calls
    actual_probs = [f"{result[0]}" for result in results]
    oracle_calls = [f"{result[1]}" for result in results]
    nums = range(1, int(np.floor((2**n) / 2)) + 1)

    noise_type = "amp"  # Define noise type
    # Define Excel filename
    
    if (n <= 7):
        excel_filename = f"./noise_{noise_type}_1_to_7_qubits.xlsx"
    else:
        excel_filename = f"./noise_{noise_type}_{n}qubits.xlsx"

    # Create DataFrames for both probability and oracle calls data
    new_column = pd.DataFrame({
        f"{noise_level} noise level": actual_probs
    })
    oracle_calls_column = pd.DataFrame({
        f"{noise_level} noise level oracle calls": oracle_calls
    })

    # Check if file exists
    if os.path.exists(excel_filename):
        try:
            # Read existing data for probabilities
            with pd.ExcelFile(excel_filename) as excel:
                # Handle probability sheet
                if f"{n} qubits" in excel.sheet_names:
                    df = pd.read_excel(excel, sheet_name=f"{n} qubits")
                    if f"{noise_level} noise level" in df.columns:
                        print(f"⚠️ Column '{noise_level} noise level' already exists - overwriting")
                        df[f"{noise_level} noise level"] = new_column
                    else:
                        df = pd.concat([df, new_column], axis=1)
                else:
                    df = pd.DataFrame({"Number of Marked Items": nums})
                    df = pd.concat([df, new_column], axis=1)
                
                # Handle oracle calls sheet
                oracle_sheet_name = f"{n}qbit_oracle"
                if oracle_sheet_name in excel.sheet_names:
                    oracle_df = pd.read_excel(excel, sheet_name=oracle_sheet_name)
                    if f"{noise_level} noise level oracle calls" in oracle_df.columns:
                        print(f"⚠️ Column '{noise_level} noise level oracle calls' already exists - overwriting")
                        oracle_df[f"{noise_level} noise level oracle calls"] = oracle_calls_column
                    else:
                        oracle_df = pd.concat([oracle_df, oracle_calls_column], axis=1)
                else:
                    oracle_df = pd.DataFrame({"Number of Marked Items": nums})
                    oracle_df = pd.concat([oracle_df, oracle_calls_column], axis=1)
        except Exception as e:
            print(f"⚠️ Error reading existing file: {e}")
            df = pd.DataFrame({"Number of Marked Items": nums})
            df = pd.concat([df, new_column], axis=1)
            oracle_df = pd.DataFrame({"Number of Marked Items": nums})
            oracle_df = pd.concat([oracle_df, oracle_calls_column], axis=1)
    else:
        # File doesn't exist - create new
        df = pd.DataFrame({"Number of Marked Items": nums})
        df = pd.concat([df, new_column], axis=1)
        oracle_df = pd.DataFrame({"Number of Marked Items": nums})
        oracle_df = pd.concat([oracle_df, oracle_calls_column], axis=1)

    # Save back to Excel
    if os.path.exists(excel_filename):
        with pd.ExcelWriter(excel_filename, 
                          engine='openpyxl', 
                          mode='a',
                          if_sheet_exists='replace') as writer:
            # Save probability data
            df.to_excel(writer, index=False, sheet_name=f"{n} qubits")
            
            # Save oracle calls data
            oracle_df.to_excel(writer, index=False, sheet_name=f"{n}qbit_oracle")
            
            # Apply number formatting to both sheets
            for sheet_name, data_frame in [(f"{n} qubits", df), (f"{n}qbit_oracle", oracle_df)]:
                worksheet = writer.sheets[sheet_name]
                # Start from column 2 (B) since column 1 is "Number of Marked Items"
                for col in range(2, len(data_frame.columns) + 1):
                    for row in range(2, len(data_frame) + 2):  # +2 for header and 1-based index
                        worksheet.cell(row=row, column=col).number_format = '0.00'
    else:
        with pd.ExcelWriter(excel_filename, 
                          engine='openpyxl', 
                          mode='w') as writer:
            # Save probability data
            df.to_excel(writer, index=False, sheet_name=f"{n} qubits")
            
            # Save oracle calls data
            oracle_df.to_excel(writer, index=False, sheet_name=f"{n}qbit_oracle")
            
            # Apply number formatting to both sheets
            for sheet_name, data_frame in [(f"{n} qubits", df), (f"{n}qbit_oracle", oracle_df)]:
                worksheet = writer.sheets[sheet_name]
                for col in range(2, len(data_frame.columns) + 1):
                    for row in range(2, len(data_frame) + 2):
                        worksheet.cell(row=row, column=col).number_format = '0.00'
    
    print(f"✅ Results saved to {excel_filename}")
    print(f"  - Probability data in sheet '{n} qubits'")
    print(f"  - Oracle calls data in sheet '{n}qbit_oracle'")

def experiment_parallel_noise(nums, n, noise_level):
    print(f"started job {nums}")

    # how many grover iterations should be performed
    iterations = grover_iterations(nums, 2**n)
    
    #fake_backend = FakeMarrakesh()
    #noise_model = NoiseModel.from_backend(fake_backend)



    noise_instance = QiskitNoiseModelAmplitudeDamp(noise_level)
    #noise_instance = QiskitNoiseModelBitflip(single_qubit_error=0.01)
    #noise_instance = QiskitNoiseModelDepol(single_qubit_error=0.01)

    noise_model = noise_instance.get_noise_model()
    sim = AerSimulator(noise_model=noise_model, shots=1000)


    iterations_per_marked_item_set = 50
    shots = 1000

    hits = 0
    for j in range(iterations_per_marked_item_set):

        marked = select_random_elements(2**n, nums)
        qc = Grover(n, marked, iterations)
        t_qc = transpile(qc, sim) 
        result = sim.run(t_qc, memory=True).result()
        counts = result.get_memory()
        for count in counts:
            found_correct_element = binary_string_in_list(marked, count)
            if found_correct_element:
                hits += 1
        print(f">>> Job {nums} finished iteration {j + 1} / {iterations_per_marked_item_set}")

    return ( ((hits / (iterations_per_marked_item_set * shots)) * 100), ((shots *iterations_per_marked_item_set)/ hits)*(iterations + 1))

#simulator_experiment(4)

#simulator_experiment_noise(3, 0.000)

if __name__ == "__main__":
    for j in range(5, 6):
        for i in range(1, 11):
            simulator_experiment_noise(j, 0.0005*i)




#TESTs: med qbits 2 till 10
# inget noise
#testa fake backend Marrakech
#noise med marrakesh's min max (0.001, 0.003) vilket vi kör mellan 0.0005 och 0.005 ta 10 steg
#gör det för både optimistisk och pessimistisk grover

#antal test 8*(1+1+10)*2 = 176

#grafer:
# alla qbits
#optimistic grover, pessimistic grover:
#(ingetnoise, marrakesh, clasikal, min, max)

# antal grafer = 8 * 2  = 16

#tabbeller för datan för noise

started job 1
>>> Job 1 finished iteration 1 / 50


KeyboardInterrupt: 